In [1]:
import re
import json
import csv
import requests
from bs4 import BeautifulSoup ,SoupStrainer
import pandas as pd 
from io import StringIO
from pathlib import Path  

!pip install selenium

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import sys

import unittest

import time


In [2]:
def get_infinity_page_html_data(chromedriver_path="./chromedriver.exe", scroll_number=30, sleep_time=2, url = ""):
    # Create selenium driver
    s = Service(chromedriver_path)
    driver = webdriver.Chrome(service=s)
    driver.get(url)

    for i in range(1,scroll_number):
        print(f"scroll_number: {i}")
        driver.execute_script("window.scrollTo(1,50000)")
        time.sleep(sleep_time)
    html_data = driver.page_source
    driver.close()
    
    soup = BeautifulSoup(html_data, 'html.parser')    
    return soup


In [3]:
def make_table(soup):
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    return df

In [4]:
def data_cleaning(df):
    df = df[df["Open"].str.contains("Dividend") == False]
    df = df[df["Open"].str.contains("Stock") == False]
    df.drop(df.tail(1).index,inplace=True)
    return df

In [5]:
def write_to_file(df, path):
    from pathlib import Path
    filepath = Path(path)
    filepath.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(filepath) 

In [6]:
base_url = "https://finance.yahoo.com/quote/"
period_url = "/history?period1=1262304000&period2=1651881600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
indexes = ["NVDA","AAPL","TSLA","AMD","MSFT","INTC"]
main_df = pd.DataFrame()

for index in indexes:
    soup = get_infinity_page_html_data(url = (base_url + index + period_url))
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    
    df = data_cleaning(df)
    
    path = index + ".csv"
    write_to_file(df, path)
    
    print()
    
   
    main_df['Date'] = df['Date']
    main_df[index + " " + "Open"] = df['Open']
    main_df[index + " " + "High"] = df['High']
    main_df[index + " " + "Low"] = df['Low']
    main_df[index + " " + "Close"] = df['Close*']
    main_df[index + " " + "Volume"] = df['Volume']
    main_df[index + " " +'Daily change'] = (((df['Close*']).astype(float))-((df['Open']).astype(float)))/((df['Open']).astype(float))*100
    main_df[index + " " +'Daily change Max'] =  abs((df['High'].astype(float)-df['Low'].astype(float))/df['Low'].astype(float)*100)
    


    
    print(df)
    print(df.info())

    
write_to_file(main_df, ("main" + ".csv"))
main_df

scroll_number: 1
scroll_number: 2
scroll_number: 3
scroll_number: 4
scroll_number: 5
scroll_number: 6
scroll_number: 7
scroll_number: 8
scroll_number: 9
scroll_number: 10
scroll_number: 11
scroll_number: 12
scroll_number: 13
scroll_number: 14
scroll_number: 15
scroll_number: 16
scroll_number: 17
scroll_number: 18
scroll_number: 19
scroll_number: 20
scroll_number: 21
scroll_number: 22
scroll_number: 23
scroll_number: 24
scroll_number: 25
scroll_number: 26
scroll_number: 27
scroll_number: 28
scroll_number: 29

              Date    Open    High     Low  Close* Adj Close**    Volume
0     May 06, 2022  187.36  195.12  179.90  186.75      186.75  63299300
1     May 05, 2022  198.67  199.25  185.00  188.44      188.44  62633100
2     May 04, 2022  199.23  204.00  187.51  203.34      203.34  64885500
3     May 03, 2022  194.00  198.25  191.33  196.02      196.02  47575100
4     May 02, 2022  185.41  195.74  183.91  195.33      195.33  57204900
...            ...     ...     ...     ...     .

scroll_number: 28
scroll_number: 29

              Date    Open    High     Low  Close* Adj Close**    Volume
0     May 06, 2022  274.81  279.25  271.27  274.73      274.73  37748300
1     May 05, 2022  285.54  286.35  274.34  277.35      277.35  43260400
2     May 04, 2022  282.59  290.88  276.73  289.98      289.98  33599300
3     May 03, 2022  283.96  284.13  280.15  281.78      281.78  25978600
4     May 02, 2022  277.71  284.94  276.22  284.47      284.47  35151100
...            ...     ...     ...     ...     ...         ...       ...
1394  Nov 21, 2016   60.50   60.97   60.42   60.86       56.47  19652600
1395  Nov 18, 2016   60.78   61.14   60.30   60.35       56.00  27686300
1396  Nov 17, 2016   60.41   60.95   59.97   60.64       56.27  32132700
1397  Nov 16, 2016   58.94   59.66   58.81   59.65       55.35  27332500
1398  Nov 15, 2016   58.33   59.49   58.32   58.87       54.62  35904100

[1378 rows x 7 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1378 entries

,Date,NVDA Open,NVDA High,NVDA Low,NVDA Close,NVDA Volume,NVDA Daily change,NVDA Daily change Max,AAPL Open,AAPL High,...,MSFT Volume,MSFT Daily change,MSFT Daily change Max,INTC Open,INTC High,INTC Low,INTC Close,INTC Volume,INTC Daily change,INTC Daily change Max
0,"May 06, 2022",187.36,195.12,179.90,186.75,63299300,-0.325576,8.460256,156.01,159.44,...,37748300,-0.029111,2.941719,44.49,45.15,43.84,44.30,40910000,-0.427062,2.988139
1,"May 05, 2022",198.67,199.25,185.00,188.44,62633100,-5.149242,7.702703,NaN,NaN,...,43260400,-2.868250,4.377779,45.60,45.91,44.13,44.60,52996700,-2.192982,4.033537
2,NaN,199.23,204.00,187.51,203.34,64885500,2.062942,8.794198,163.85,164.08,...,33599300,2.615096,5.113287,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"May 04, 2022",194.00,198.25,191.33,196.02,47575100,1.041237,3.616788,159.67,166.48,...,25978600,-0.767714,1.420667,45.06,46.64,44.41,46.54,44625600,3.284510,5.021392
4,"May 03, 2022",185.41,195.74,183.91,195.33,57204900,5.350305,6.432494,158.15,160.71,...,35151100,2.434194,3.156904,44.79,45.49,44.23,45.06,40158400,0.602813,2.848745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,"Nov 21, 2016",23.33,23.43,23.09,23.41,33036800,0.342906,1.472499,27.99,28.10,...,27686300,-0.707470,1.393035,35.10,35.20,34.84,34.98,14259500,-0.341880,1.033295
1396,"Nov 18, 2016",23.52,23.59,23.10,23.25,43643600,-1.147959,2.121212,27.53,28.00,...,32132700,0.380732,1.634150,34.90,35.00,34.64,34.95,16806900,0.143266,1.039261
1397,"Nov 17, 2016",23.10,23.58,22.95,23.34,57606000,1.038961,2.745098,27.43,27.64,...,27332500,1.204615,1.445332,34.81,35.02,34.63,35.02,15748500,0.603275,1.126191
1398,"Nov 16, 2016",23.08,23.70,22.66,23.10,83298800,0.086655,4.589585,27.45,27.59,...,35904100,0.925767,2.006173,34.90,34.92,34.54,34.84,18565500,-0.171920,1.100174


In [7]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1377 entries, 0 to 1399
Data columns (total 43 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   1355 non-null   object 
 1   NVDA Open              1377 non-null   object 
 2   NVDA High              1377 non-null   object 
 3   NVDA Low               1377 non-null   object 
 4   NVDA Close             1377 non-null   object 
 5   NVDA Volume            1377 non-null   object 
 6   NVDA Daily change      1377 non-null   float64
 7   NVDA Daily change Max  1377 non-null   float64
 8   AAPL Open              1354 non-null   object 
 9   AAPL High              1354 non-null   object 
 10  AAPL Low               1354 non-null   object 
 11  AAPL Close             1354 non-null   object 
 12  AAPL Volume            1354 non-null   object 
 13  AAPL Daily change      1354 non-null   float64
 14  AAPL Daily change Max  1354 non-null   float64
 15  TSLA